In [1]:
# Import necessary libraries
import numpy as np
import pycuda.autoinit                                                  # type: ignore
import pycuda.driver as cuda                                            # type: ignore
from pycuda import gpuarray, compiler                                   # type: ignore
from osgeo import gdal                                                  # type: ignore
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter                               # type: ignore
from datetime import datetime
import zipfile
import os

In [2]:
# Helper function to print available CUDA devices
def printGPU():
    """
    Print information about available CUDA devices.
    """
    num_devices = cuda.Device.count()
    print(f"{num_devices} device(s) found.")
    for ordinal in range(num_devices):
        dev = cuda.Device(ordinal)
        print(f"Device #{ordinal}: {dev.name()}")
    print(cuda)

In [3]:
def svfCalculator_RayTracingOnGPU(dsm, vegdem, vegdem2, scale, transparency_factor, svf_type, theta_start, theta_end, traceRadius, numDirections):
    """
    Calculate Sky View Factor (SVF) using ray tracing on a GPU.

    Parameters:
        dsm (np.ndarray): Digital Surface Model.
        vegdem (np.ndarray): Vegetation height model.
        vegdem2 (np.ndarray): Vegetation canopy bottom height model.
        scale (float): Scale factor for the height.
        transparency_factor (float): Factor representing the transparency of vegetation.
        svf_type (int): Type of SVF calculation (0: DSM only, 1: Veg, 2: Veg & DSM).
        theta_start (int): Starting angle for theta in degrees.
        theta_end (int): Ending angle for theta in degrees.

    Returns:
        np.ndarray: Calculated Sky View Factor as a 2D array.
    """
    # Convert input arrays to float32
    px = np.array(dsm, dtype=np.float32)
    veg_px = np.array(vegdem, dtype=np.float32)
    veg_px2 = np.array(vegdem2, dtype=np.float32)
    height, width = px.shape

    # Allocate GPU memory
    d_px = gpuarray.to_gpu(px)
    d_veg_px = gpuarray.to_gpu(veg_px)
    d_veg_px2 = gpuarray.to_gpu(veg_px2)
    d_out = gpuarray.empty((height, width), np.float32)


    # CUDA kernel code for SVF calculation
    kernel_code = """
    #include <math.h>
    #define PI 3.141592654

    __global__ void svfcalculator(float *lattice_out, float *lattice, float *veg_lattice, float *veg_lattice2, 
                                float scale, float transparency_factor, int width, int height, 
                                int svf_type, int theta_start, int theta_end, int numDirections, float traceRadius) {
        int ix = threadIdx.x + blockIdx.x * blockDim.x;
        int iy = threadIdx.y + blockIdx.y * blockDim.y;
        if (ix >= width || iy >= height) return;

        int index = ix + iy * width;
        float clr00 = lattice[index];
        float free_fraction_total = 0.0;

        // Precompute theta values based on numDirections
        float theta[360];  // Array to hold theta values
        for (int t = 0; t < numDirections; ++t) {
            theta[t] = PI * t / numDirections;  // Use numDirections for division
        }

        // Ensure theta_start and theta_end are valid (between 0 and numDirections)
        int adjusted_theta_start = theta_start % numDirections;
        int adjusted_theta_end = theta_end % numDirections;

        // Loop over theta in the specified range, considering wraparound for angles crossing 0
        int theta_range = (adjusted_theta_start < adjusted_theta_end) ? 
                        (adjusted_theta_end - adjusted_theta_start) : 
                        (numDirections - adjusted_theta_start + adjusted_theta_end);

        for (int thetaN = 0; thetaN < theta_range; ++thetaN) {
            int current_theta = (adjusted_theta_start + thetaN) % numDirections;
            float free_fraction = 1.0f;
            bool gamma_stop = false;

            int gamma_steps = 90;  // Same number of gamma steps
            float gamma_step_inv = 1.0f / float(gamma_steps);

            // Precompute gamma values
            float gamma[91];
            for (int g = 0; g <= gamma_steps; ++g) {
                gamma[g] = (PI / 2.0f) * g * gamma_step_inv;
            }         

            for (int gammaN = 0; gammaN <= gamma_steps; gammaN++) {
                float radius = 1.0f;
                float cos_theta = cosf(theta[current_theta]);
                float sin_theta = sinf(theta[current_theta]); 
                bool crossed_veg = false;          

                // Trace the ray until traceRadius
                while (radius < traceRadius) {  // Replace 449.0 with traceRadius
                    int x = ix + int(radius * cos_theta);
                    int y = iy - int(radius * sin_theta);
                    if (x < 0 || x >= width || y < 0 || y >= height) break;
                    int index2 = x + y * width;                    

                    float dsm_height = lattice[index2];
                    float veg_height = veg_lattice[index2];
                    float canopy_bottom_height = veg_lattice2[index2];
                    float ray_height = clr00 + radius * tanf(gamma[gammaN]);                    

                    if (svf_type == 0){
                        if (ray_height < dsm_height){
                            free_fraction -= gamma_step_inv;
                            break;
                        } else if (radius == traceRadius){  // Replace 449.0 with traceRadius
                            gamma_stop = true;                            
                        } 
                    }
                    if (svf_type == 1){
                        if(ray_height > canopy_bottom_height && ray_height < veg_height){
                            free_fraction -= (transparency_factor * gamma_step_inv);
                            break;
                        } else if (radius == traceRadius){  // Replace 449.0 with traceRadius
                            gamma_stop = true;
                        }
                    }
                    if (svf_type == 2) {
                        if (ray_height < dsm_height && !crossed_veg) break;
                        else if (ray_height > canopy_bottom_height && ray_height < veg_height) crossed_veg = true;
                        else if (ray_height < dsm_height && crossed_veg){
                            free_fraction -= (transparency_factor * gamma_step_inv);
                            break;
                        }
                        else if (radius == traceRadius) gamma_stop = true;  // Replace 449.0 with traceRadius
                    }
                    radius += 1.0f;
                }
                if (gamma_stop) break;
            }
            free_fraction_total += free_fraction;
        }

        lattice_out[index] = free_fraction_total / float(theta_range);
    }
    """

    # Compile the CUDA kernel
    mod = compiler.SourceModule(kernel_code)
    svf_kernel = mod.get_function("svfcalculator")

    # Define block and grid sizes
    block_size = (32, 32, 1)
    grid_size = (int(np.ceil(width / block_size[0])), int(np.ceil(height / block_size[1])))

    # Launch the kernel
    svf_kernel(
        d_out, d_px, d_veg_px, d_veg_px2, np.float32(scale), np.float32(transparency_factor),
        np.int32(width), np.int32(height), np.int32(svf_type),
        np.int32(theta_start), np.int32(theta_end), np.int32(numDirections), np.float32(traceRadius),
        block=block_size, grid=grid_size
    )
    
    
    # Synchronize the CUDA context
    cuda.Context.synchronize()
    return d_out.get()

In [4]:
# Main function for processing DSM and vegetation data
def main(dsm_path, vegdem_path, vegdem2_path, transparency_factor, numDirections, traceRadius, output_dir):
    """
    Main function to calculate SVF from DSM and vegetation models.

    Parameters:
        dsm_path (str): Path to the DSM raster.
        vegdem_path (str): Path to the vegetation height raster.
        vegdem2_path (str): Path to the vegetation canopy bottom height raster.
        transparency_factor (float): Transparency factor for vegetation.
        output_dir (str): Directory for saving output files.
    """


    # Load DSM and vegetation rasters
    dsm_ds = gdal.Open(dsm_path)
    vegdem_ds = gdal.Open(vegdem_path)
    vegdem2_ds = gdal.Open(vegdem2_path)

    # Potentially rescaling the resolution to match 1mx1m
    geotransform = dsm_ds.GetGeoTransform()
    #scale = 1 / geotransform[1]                            # Rescale the resolution to 1mx1m
    scale = 1                                             # Keep the resolution as the input DSM  

    # Read arrays from raster files
    dsm = dsm_ds.ReadAsArray().astype(np.float32)
    vegdem = vegdem_ds.ReadAsArray().astype(np.float32)
    vegdem2 = vegdem2_ds.ReadAsArray().astype(np.float32)

    # Adjust vegetation heights by adding DSM height
    vegdem = vegdem + dsm
    vegdem2 = vegdem * 0.25 + dsm
    #vegdem2 = vegdem2 + dsm

    # Apply Gaussian filter for smoothing
    vegdem = gaussian_filter(vegdem, sigma=0.5)
    vegdem2 = gaussian_filter(vegdem2, sigma=0.5)

    driver = gdal.GetDriverByName("GTiff")

    # Helper function to save SVF results to GeoTIFF
    def save_svf(filename, svf_data):
        out_ds = driver.Create(
            filename, dsm.shape[1], dsm.shape[0], 1, gdal.GDT_Float32
        )
        out_ds.GetRasterBand(1).WriteArray(svf_data)
        out_ds.SetGeoTransform(dsm_ds.GetGeoTransform())
        out_ds.SetProjection(dsm_ds.GetProjection())
        out_ds.FlushCache()

    # Helper function to plot SVF data
    def plot_svf(svf_data, title):
        plt.figure(figsize=(5, 3))
        plt.imshow(svf_data, cmap="viridis")
        plt.colorbar(label="SVF Value")
        plt.title(title)
        plt.show()

    # Directions for theta range (Wind directions: W, S, E, N)
    directions = {
        "E": (0, 180),
        'S': (90, 270),
        'W': (180, 360),
        'N': (270, 90)
         }
    
    #Update the directions from 360 (degrees) to numDirections
    divisor = 360/numDirections    
    directions = {key: (value[0] / divisor, value[1] / divisor) for key, value in directions.items()}
    
    # Prepare arrays to store the sums for calculating averages
    svf_mean = np.zeros_like(dsm, dtype=np.float32)
    svfveg_mean = np.zeros_like(dsm, dtype=np.float32)
    svfaveg_mean = np.zeros_like(dsm, dtype=np.float32)

    # Prepare a list to hold the file paths for zipping
    file_paths = []

    timestamp_start = datetime.now()
    print (f"Started SVF-calculations at {timestamp_start.strftime('%H:%M:%S')}")

    for direction, (theta_start, theta_end) in directions.items():
        timestamp_startdir = datetime.now()

        # Calculate different types of SVF
        svf = svfCalculator_RayTracingOnGPU(dsm, vegdem, vegdem2, scale, transparency_factor, 0, theta_start, theta_end, traceRadius, numDirections)
        svfveg = svfCalculator_RayTracingOnGPU(dsm, vegdem, vegdem2, scale, transparency_factor, 1, theta_start, theta_end, traceRadius, numDirections)
        svfaveg = svfCalculator_RayTracingOnGPU(dsm, vegdem, vegdem2, scale, transparency_factor, 2, theta_start, theta_end, traceRadius, numDirections)

        # Accumulate the SVF values for averaging later
        svf_mean += svf
        svfveg_mean += svfveg
        svfaveg_mean += svfaveg
        
        # Define file names based on svf_type
        svf_files = [
            (svf, f"{output_dir}/svf{direction}.tif"),              # svf_type == 0  == dsm only
            (svfveg, f"{output_dir}/svf{direction}veg.tif"),        # svf_type == 1  == vegetation only
            (svfaveg, f"{output_dir}/svf{direction}aveg.tif")       # svf_type == 2  == vegetation on buildings
        ]

        # Plot the SVF files
        for svf_data, filename in svf_files:
            plot_svf(svf_data, f"{filename}")
  
        # Save the SVF files and collect the file paths
        for svf_data, filename in svf_files:
            save_svf(filename, svf_data)
            file_paths.append(filename)
        
        timestamp_gen = datetime.now()
        time_proc = timestamp_gen-timestamp_startdir
        time_proc_sec = time_proc.total_seconds()
        proc_minutes, proc_seconds = divmod(time_proc_sec,60)

        print (f"Generated svf-files for direction {direction} at {timestamp_gen.strftime('%H:%M:%S')}")
        print (f"Processing time for direction {direction} : {int(proc_minutes)} minutes and {proc_seconds:.2f} seconds")

    # Total processing time calculations
    timestamp_end = datetime.now()
    time_proc_tot = timestamp_end - timestamp_start
    time_proc_tot_sec = time_proc_tot.total_seconds()
    proc_tot_minutes, proc_tot_seconds = divmod(time_proc_tot_sec,60)

    print (f"Total processing time for all directions : {int(proc_tot_minutes)} minutes and {proc_tot_seconds:.2f} seconds")

    # Calculate the average SVF by dividing by the number of directions
    num_directions = len(directions)
    svf_mean /= num_directions
    svfveg_mean /= num_directions
    svfaveg_mean /= num_directions

    # Save the averaged SVF results
    save_svf(f"{output_dir}/svf.tif", svf_mean)
    save_svf(f"{output_dir}/svfveg.tif", svfveg_mean)
    save_svf(f"{output_dir}/svfaveg.tif", svfaveg_mean)

    # Add the averaged files to the list of files to be zipped
    file_paths.extend([
        f"{output_dir}/svf.tif",
        f"{output_dir}/svfveg.tif",
        f"{output_dir}/svfaveg.tif"
    ])
    
    # Zip all the generated files
    zip_filename = f"{output_dir}/svfs_GPU.zip"
    with zipfile.ZipFile(zip_filename, 'w') as zipf:
        for file in file_paths:
            zipf.write(file, os.path.basename(file))                

    # Delete the original individual files (not in .zip)
    for file in file_paths:
        os.remove(file)

    # Close datasets
    dsm_ds = vegdem_ds = vegdem2_ds = None

In [ ]:
# Run main function
if __name__ == "__main__":
    dsm_path            = "C:/Users/Max/Documents/Werk/data_spatial/37HN1_09/GPU_test/DSM.tif"              # DSM raster file path
    vegdem_path         = "C:/Users/Max/Documents/Werk/data_spatial/37HN1_09/GPU_test/CDSM.tif"             # CDSM raster file path
    ###check if vegdem2 is used or that TDSM is set to a percentage of vegdem in main-function
    vegdem2_path        = "C:/Users/Max/Documents/Werk/data_spatial/37HN1_09/TDSM_1m_37HN1_09.tif"          # TDSM raster file path
    transparency_factor =  0.97                                                                             # Transparency/opacity factor for vegetation impact
    output_dir          = "C:/Users/Max/Documents/Werk/test_gpu/output"                                     # Output directory
    numDirections       =  16                                                                               # Number of directions to check for SVF
    traceRadius         =  200                                                                              # Radius to check for obstructions in dsm resolution/meters (depending on scale settings)
    gdal.DontUseExceptions()                                                                                # Suppress warnings from GDAL

    main(dsm_path, vegdem_path, vegdem2_path, transparency_factor, numDirections, traceRadius, output_dir)